In [1]:
import math
import numpy as np
import scipy.special as spf
import vegas # numeric integration
import gvar # gaussian variables; for vegas
import time
import quaternionic # For rotations
import spherical #For Wigner D matrix
# import csv # file IO for projectFnlm
# import os.path
import h5py # database format for mathcalI arrays
import importlib
import sys
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import numba


sys.path.append('../')

import vsdm
from vsdm.units import *
from vsdm.utilities import *
vsdm.__version__

'0.4.0'

In [2]:
"""
    Defining the momentum form factor for the particle-in-a-box model.

    "model 4": rectangular box with dimensions (4 a0, 7 a0, 10 a0) 
        with a0 the Bohr radius
    default transition: from ground state to the n_{x,y,z} = (1, 1, 2) excited state
    "alt" transition: from ground state to the n = (3, 2, 1) excited state. 

    Both form factors are symmetric under reflections in the x, y, or z directions. 
"""

@numba.jit("double(int64,double)", nopython=True)
def fj2(nj, qLj):
    if qLj==0:
        if nj==1:
            return 1
        else:
            return 0
    qlp = np.abs(qLj)/np.pi
    # mathsinc(x) = np.sinc(x/pi)
    s_minus = np.sinc(0.5*(qlp - nj + 1))/(1 + (nj-1)/qlp)
    s_plus = np.sinc(0.5*(qlp - nj - 1))/(1 + (nj+1)/qlp)
    return (s_minus + s_plus)**2

# Long thin box limit: assuming that Lz > Lx,Ly,
# so the lowest excited states are nz=2, nz=3, with nx=ny=1.

@numba.jit("double(double[:],int64,double[:])", nopython=True)
def fs2_nz(Lvec, nz, q_xyz):
    # q: the DM particle velocity (cartesian, lab frame)
    # L: the dimensions of the box
    # nz = 2, 3, 4... The final state. (n=1 defined as ground state)
    # fs2 is dimensionless
    # note: np.sinc(x/pi) = sin(x) / (x). Included in defs. of qL below
    [Lx, Ly, Lz] = Lvec
    [qx, qy, qz] = q_xyz
    qLx = Lx*qx
    qLy = Ly*qy
    qLz = Lz*qz
#     qL = qLx + qLy + qLz
    fx2 = fj2(1, qLx)
    fy2 = fj2(1, qLy)
    fz2 = fj2(nz, qLz)
    return fx2*fy2*fz2

@numba.jit("double(double[:],int64[:],double[:])", nopython=True)
def fs2_nxyz(Lvec, n_xyz, q_xyz):
    # q: the DM particle velocity (cartesian, lab frame)
    # L: the dimensions of the box
    # nz = 2, 3, 4... The final state. (n=1 defined as ground state)
    # fs2 is dimensionless
    # note: np.sinc(x/pi) = sin(x) / (x). Included in defs. of qL below
    [Lx, Ly, Lz] = Lvec
    [qx, qy, qz] = q_xyz
    [nx, ny, nz] = n_xyz
    qLx = Lx*qx
    qLy = Ly*qy
    qLz = Lz*qz
    fx2 = fj2(nx, qLx)
    fy2 = fj2(ny, qLy)
    fz2 = fj2(nz, qLz)
    return fx2*fy2*fz2

@numba.jit("double(int64,double)", nopython=True)
def DeltaE_nz(nz, Lz):
    # for nx=ny=1 final states, in units of [q**2]/mElec
    return 0.5*math.pi**2 / mElec * (nz**2 - 1)/Lz**2

# Cartesian version of fs2:
@numba.jit("double(double[:])", nopython=True)
def fs2_model4_cart(q_xyz):
    return fs2_nz(np.array([4/qBohr, 7/qBohr, 10/qBohr]), 2, q_xyz)

# Cartesian version of fs2:
@numba.jit("double(double[:])", nopython=True)
def fs2_model4_cart_alt(q_xyz):
    Lvec = np.array([4/qBohr, 7/qBohr, 10/qBohr])
    n_xyz = np.array([3, 2, 1])
    return fs2_nxyz(Lvec, n_xyz, q_xyz)


### Defining the function fs2(qSph) that EvaluateFnlm will use as the input.
#   The function is decorated with symmetry identifiers (phi_even, phi_cyclic, etc)
#   This function is not a GaussianF instance, a sum of gaussians (is_gaussian==False) 

QMAX = 10*qBohr # Global value for q0=qMax for wavelets

# @numba.jit("double(double[:])", nopython=True)
def fs2_model4(qSph):
    [q, theta, phi] = qSph
    qx = q*math.sin(theta) * math.cos(phi)
    qy = q*math.sin(theta) * math.sin(phi)
    qz = q*math.cos(theta)
    q_xyz = np.array([qx, qy, qz])
    Lvec = np.array([4/qBohr, 7/qBohr, 10/qBohr])
    return fs2_nz(Lvec, 2, q_xyz)
fs2_model4.is_gaussian = False
fs2_model4.z_even = True
fs2_model4.phi_even = True
fs2_model4.phi_cyclic = 2
fs2_model4.center_Z2 = True
fs2_model4.DeltaE = 4.03*eV

def fs2_model4_alt(qSph):
    [q, theta, phi] = qSph
    qx = q*math.sin(theta) * math.cos(phi)
    qy = q*math.sin(theta) * math.sin(phi)
    qz = q*math.cos(theta)
    q_xyz = np.array([qx, qy, qz])
    return fs2_model4_cart_alt(q_xyz)
fs2_model4_alt.is_gaussian = False
fs2_model4_alt.z_even = True
fs2_model4_alt.phi_even = True
fs2_model4_alt.phi_cyclic = 2

In [3]:
### MOMENTUM DISTRIBUTION EXAMPLE
QMAX = 10*qBohr # Global value for q0=qMax for wavelets

Qdict = dict(u0=QMAX, type='wavelet', uMax=QMAX)

# # Read Fnlm from saved csv file...
fs2_csv = 'demo_fs2'
fs2 = vsdm.Fnlm(Qdict, f_type='fs2', use_gvar=False)
fs2.center_Z2 = True
fs2.importFnlm_csv('./tools/demo/demo_fs2.csv')
print(fs2.basis)
print('t_eval:', fs2.t_eval)
print('nCoeffs = {}'.format(len(fs2.f_nlm.keys())))

### VELOCITY DISTRIBUTION EXAMPLE
# Model 4: a bunch of streams, not symmetric. 
# Including the halo component without vEsc.

VMAX = 960.*km_s # Global value for v0=vMax for wavelets
Vdict = dict(u0=VMAX, type='wavelet', uMax=VMAX)

"""Read from CSV"""
gX_csv = 'gX_model4'
gX = vsdm.Fnlm(Vdict, f_type='gX', use_gvar=False)
gX.importFnlm_csv('./tools/demo/gX_model4.csv')
print(gX.basis)
print('t_eval:', gX.t_eval)
print('nCoeffs = {}'.format(len(gX.f_nlm.keys())))

{'u0': 37289.47137978341, 'type': 'wavelet', 'uMax': 37289.47137978341}
t_eval: 0.33824682235717773
nCoeffs = 114688
{'u0': 0.0032022202060095, 'type': 'wavelet', 'uMax': 0.0032022202060095}
t_eval: 4.353804111480713
nCoeffs = 350464


In [4]:
n = 300  # for precision goal of 0.3% 
# ellMax = 24
ellMax = 12
nvMax = 127
nqMax = 127

# ls10 = [1, 1.5, 2, 3, 4, 5, 6, 7, 8, 9]
ls10 = [1]
p10 = [1, 10, 100]
mXlist = np.array([p*m for p in p10 for m in ls10])

"""Check the evaluation time for mcalI:"""
mI = {}
modelsDM = []
t0 = time.time()
for fn in [0,2]:
    for mX in mXlist:
        modelsDM += [(mX, fn)]
        dmModel = dict(mX=mX*MeV, fdm_n=fn, mSM=mElec, DeltaE=fs2_model4.DeltaE)
        mI[(mX, fn)] = vsdm.McalI(Vdict, Qdict, dmModel, 
                                  mI_shape=(ellMax+1, nvMax+1,nqMax+1), center_Z2=True, 
                                  use_gvar=False, do_mcalI=True)
        print('\t', (mX, fn), ": ", mI[(mX, fn)].t_eval)
tEvalI = time.time() - t0
print('tI avg:', tEvalI/len(modelsDM))

	 (1, 0) :  2.237488031387329
	 (10, 0) :  17.831483840942383
	 (100, 0) :  23.482996940612793
	 (1, 2) :  3.0485970973968506
	 (10, 2) :  18.40968608856201
	 (100, 2) :  23.67629909515381
tI avg: 14.781431674957275


In [5]:
rates={}
for DM in mI.keys():
    rates[DM]=vsdm.RateCalc(gX, fs2, mI[DM], 
                    use_gvar=False, sparse=False)
print(rates)

TypeError: 'NoneType' object cannot be interpreted as an integer

In [ ]:
rates[(1,0)].vecK